# spiral_builder

In [ ]:
open file_system_operators
open rust_operators
open sm'_operators

In [ ]:
//// test

open testing

## types

In [ ]:
inl types () =
    env.types ()
    file_system.types ()
    runtime.types ()
    rust.types ()
    sm'.types ()

## get_args

In [ ]:
inl get_args () =
    {
        fsharp = "fsharp", {
            path = "path", 'p'
            package_dir = "package-dir", 'd'
            args = "args", 'a'
        }
        rust = "rust", {
            deps = "deps", 'd'
        }
        dib = "dib", {
            path = "path", 'p'
            retries = "retries", 'r'
            working_directory = "working_directory", 'w'
        }
        trace_level = "trace-level", 't'
    }

## get_command

In [ ]:
let get_command () =
    ##"command"
    |> runtime.new_command
    |> runtime.command_init_arg (get_args () .trace_level) (
        real runtime.arg_union `trace_level ignore
    )
    |> runtime.command_subcommand_required true
    |> runtime.command_subcommand (
        ##(get_args () .fsharp |> fst)
        |> runtime.new_command
        |> runtime.command_init_arg ((get_args () .fsharp |> snd).path) (
            runtime.arg_required true
        )
        |> runtime.command_init_arg ((get_args () .fsharp |> snd).package_dir) (
            runtime.arg_required true
        )
        |> runtime.command_init_arg ((get_args () .fsharp |> snd).args) (
            id
        )
    )
    |> runtime.command_subcommand (
        ##(get_args () .rust |> fst)
        |> runtime.new_command
        |> runtime.command_init_arg ((get_args () .rust |> snd).deps) (
            runtime.arg_value_names ;[ ##"NAME"; ##"VERSION" ]
            >> runtime.arg_num_args_range (
                runtime.new_value_range
                    (am'.Start (1i32 |> unativeint))
                    (am'.End id)
            )
            >> runtime.arg_action runtime.Append
        )
    )
    |> runtime.command_subcommand (
        ##(get_args () .dib |> fst)
        |> runtime.new_command
        |> runtime.command_init_arg ((get_args () .dib |> snd).path) (
            runtime.arg_required true
            // >> runtime.arg_value_parser (runtime.value_parser_path_buf ())
        )
        |> runtime.command_init_arg ((get_args () .dib |> snd).retries) (
            runtime.arg_value_parser (runtime.value_parser_expr "u8")
        )
        |> runtime.command_init_arg ((get_args () .dib |> snd).working_directory) (
            id
        )
    )

## get_workspace_cargo_toml_content

In [ ]:
inl get_workspace_cargo_toml_content () : string =
    $'$"""[workspace]\nresolver = "2"\nmembers = ["packages/*"]\n\n[workspace.dependencies]\nfable_library_rust = {{ path = \"../../../../../lib/rust/fable/fable_modules/fable-library-rust\", default-features = false, features = [\"static_do_bindings\", \"datetime\", \"guid\", \"threaded\"] }}\ninline_colorization = \"~0.1\"\n\n"""'

## get_cargo_toml_content

In [ ]:
inl get_cargo_toml_content { hash deps } : string =
    $'$"""[package]\nname = \"spiral_builder_{!hash}\"\nversion = "0.0.1"\nedition = "2021"\n\n[dependencies]\nfable_library_rust = {{ workspace = true }}\ninline_colorization = {{ workspace = true }}\n{!deps}\n\n[[bin]]\nname = \"spiral_builder_{!hash}\"\npath = \"spiral_eval.rs\" """'

## process_dib

In [ ]:
inl process_dib { path retries working_directory } =
    inl exit_code, repl_result =
        let rec loop retry =
            inl exit_code, repl_result =
                runtime.execution_options fun x => { x with
                    command = $'$"dotnet repl --exit-after-run --run \\\"{!path}\\\" --output-path \\\"{!path}.ipynb\\\""'
                    environment_variables = ;[ "AUTOMATION", "True" ]
                    trace = false
                    working_directory
                }
                |> runtime.execute_with_options
            
            if exit_code = 0 || retry >= retries
            then exit_code, repl_result
            else
                trace Debug
                    fun () => $'"spiral_builder.run / repl error"'
                    fun () => $'$"exit_code: {!exit_code} / retry: {!retry}/{!retries} / repl_result: {!repl_result} / {!_locals ()}"'
                loop (retry + 1)
        loop 1

    inl exit_code, result =
        if exit_code <>. 0
        then exit_code, repl_result
        else
            inl exit_code, jupyter_result =
                runtime.execution_options fun x => { x with
                    command = $'$"jupyter nbconvert \\\"{!path}.ipynb\\\" --to html --HTMLExporter.theme=dark"'
                }
                |> runtime.execute_with_options

            trace Debug
                fun () => $'"spiral_builder.run / dib / jupyter nbconvert"'
                fun () => $'$"exit_code: {!exit_code} / jupyter_result.Length: {!jupyter_result.Length} / {!_locals ()}"'

            if exit_code <>. 0
            then exit_code, $'$"repl_result: {!repl_result}\n\njupyter_result: {!jupyter_result}"'
            else
                inl exit_code, pwsh_replace_html_result =
                    inl path = path |> sm'.replace "'" "''"
                    runtime.execution_options fun x => { x with
                        command = $'$"pwsh -c \\\"$counter = 1; $path = \'{!path}.html\'; (Get-Content $path -Raw) -replace \'(id=\\\\\\"cell-id=)[a-fA-F0-9]{{8}}\', {{ $_.Groups[1].Value + $counter++ }} | Set-Content $path\\\""'
                    }
                    |> runtime.execute_with_options

                trace Debug
                    fun () => $'"spiral_builder.run / dib / html cell ids"'
                    fun () => $'$"exit_code: {!exit_code} / pwsh_replace_html_result.Length: {!pwsh_replace_html_result.Length} / {!_locals ()}"'

                $'$"{!path}.html"'
                |> file_system.read_all_text
                |> sm'.replace "\r\n" "\n"
                |> file_system.write_all_text $'$"{!path}.html"'

                $'$"{!path}.ipynb"'
                |> file_system.read_all_text
                |> sm'.replace "\\r\\n" "\\n"
                |> file_system.write_all_text $'$"{!path}.ipynb"'

                exit_code, $'$"repl_result: {!repl_result}\n\njupyter_result: {!jupyter_result}\n\npwsh_replace_html_result: {!pwsh_replace_html_result}"'

    trace Debug
        fun () => $'"spiral_builder.run / dib"'
        fun () => $'$"exit_code: {!exit_code} / result.Length: {!result.Length} / {!_locals ()}"'

    if exit_code <>. 0
    then failwith $'$"spiral_builder.run / dib / exit_code: {!exit_code} / result: {!result} / {!_locals ()}"'
    ;[
        "stdio",
        result
    ]

## process_fsharp

In [ ]:
inl process_fsharp { path package_dir deps } =
    inl path = path |> file_system.get_full_path
    inl package_dir = package_dir |> file_system.get_full_path
    inl hash = package_dir |> file_system.get_file_name

    inl deps =
        deps
        |> am'.vec_map fun dep =>
            inl dep = dep |> sm'.from_std_string
            if dep |> sm'.contains "="
            then dep
            else $'$"{!dep}=\\"*\\""'
        |> am'.from_vec
        |> fun x => x : _ i32 _
        |> seq.of_array'
        |> sm'.concat "\n"

    ;[
        "cargo_toml_content",
        get_cargo_toml_content { hash deps }

        "workspace_cargo_toml_content",
        get_workspace_cargo_toml_content ()
    ]

## run

In [ ]:
let rec run (matches : runtime.arg_matches) : async.future_pin (resultm.result' string string) =
    fun () =>
        match matches |> runtime.matches_subcommand |> optionm'.unbox with
        | Some (subcommand, arg_matches)
                when sm'.from_std_string subcommand = (get_args () .fsharp |> fst) =>
            inl path =
                arg_matches
                |> runtime.matches_get_one ((get_args () .fsharp |> snd).path |> fst)
                |> optionm'.unbox
                |> optionm.value
                |> sm'.from_std_string

            inl package_dir =
                arg_matches
                |> runtime.matches_get_one ((get_args () .fsharp |> snd).package_dir |> fst)
                |> optionm'.unbox
                |> optionm.value
                |> sm'.from_std_string

            inl args =
                arg_matches
                |> runtime.matches_get_one ((get_args () .fsharp |> snd).args |> fst)
                |> optionm'.unbox
                |> optionm.map sm'.from_std_string
            
            inl result =
                match args with
                | Some args =>
                    get_command ()
                    |> runtime.command_get_matches_from ($'$"_ {!args}"' |> runtime.split_args)
                    |> run
                    |> async.block_on
                    |> resultm.unwrap'
                | None => "{}"
                |> sm'.deserialize
                |> resultm.unwrap'
            
            inl deps =
                result
                |> mapm.get ("deps" |> sm'.to_std_string)
                |> optionm'.unwrap
                |> sm'.from_std_string
                |> sm'.deserialize
                |> resultm.unwrap'

            process_fsharp { path package_dir deps }

        | Some (subcommand, arg_matches)
                when sm'.from_std_string subcommand = (get_args () .rust |> fst) =>
            inl deps : am'.vec sm'.std_string =
                arg_matches
                |> runtime.matches_get_many ((get_args () .rust |> snd).deps |> fst)
                |> optionm'.unbox
                |> optionm'.default_value (;[] |> am'.to_vec)

            ;[
                "deps",
                deps |> sm'.serialize |> resultm.unwrap' |> sm'.from_std_string
            ]

        | Some (subcommand, arg_matches)
            when sm'.from_std_string subcommand = (get_args () .dib |> fst)
            =>
            inl path =
                arg_matches
                |> runtime.matches_get_one ((get_args () .dib |> snd).path |> fst)
                |> optionm'.map'' (
                    sm'.from_std_string
                    >> file_system.absolute_path
                )
                |> optionm'.unwrap

            inl retries =
                arg_matches
                |> runtime.matches_get_one ((get_args () .dib |> snd).retries |> fst)
                |> optionm'.default_value' 1u8

            inl working_directory : optionm'.option' string =
                arg_matches
                |> runtime.matches_get_one ((get_args () .dib |> snd).working_directory |> fst)

            process_dib { path retries working_directory }

        | Some (subcommand, arg_matches) =>
            trace Debug
                fun () => $'"spiral_builder.run"'
                fun () => $'$"subcommand: {!subcommand} / {!_locals ()}"'

            ;[]
        | _ => ;[]
        |> am'.to_vec
        |> am'.vec_map' fun k, v =>
            new_pair (sm'.to_std_string k) (sm'.to_std_string v)
        |> mapm.b_tree_map_from_vec_pairs
        |> sm'.serialize
        |> resultm.map_error' (sm'.format' >> sm'.from_std_string)
        |> resultm.map' sm'.from_std_string
    |> async.future_init (3, 2) 1

In [ ]:
//// test
///! rust -d clap encoding encoding_rs encoding_rs_io futures-lite regex serde_json

types ()
get_command ()
|> runtime.command_get_matches_from ($'$"_ fsharp -p p -d d -a rust"' |> runtime.split_args)
|> run
|> async.block_on
|> resultm.unwrap'
|> _assert_eq (
    ;[
        "cargo_toml_content",
        get_cargo_toml_content { hash = "d"; deps = "" }

        "workspace_cargo_toml_content",
        get_workspace_cargo_toml_content ()
    ]
    |> am'.to_vec
    |> am'.vec_map' fun k, v =>
        new_pair (sm'.to_std_string k) (sm'.to_std_string v)
    |> mapm.b_tree_map_from_vec_pairs
    |> sm'.serialize
    |> resultm.unwrap'
    |> sm'.from_std_string
)

assert_eq / actual: "{"cargo_toml_content":"[package]\nname = \"spiral_builder_d\"\nversion = \"0.0.1\"\nedition = \"2021\"\n\n[dependencies]\nfable_library_rust = { workspace = true }\ninline_colorization = { workspace = true }\n\n\n[[bin]]\nname = \"spiral_builder_d\"\npath = \"spiral_eval.rs\" ","workspace_cargo_toml_content":"[workspace]\nresolver = \"2\"\nmembers = [\"packages/*\"]\n\n[workspace.dependencies]\nfable_library_rust = { path = \"../../../../../lib/rust/fable/fable_modules/fable-library-rust\", default-features = false, features = [\"static_do_bindings\", \"datetime\", \"guid\", \"threaded\"] }\ninline_colorization = \"~0.1\"\n\n"}" / expected: "{"cargo_toml_content":"[package]\nname = \"spiral_builder_d\"\nversion = \"0.0.1\"\nedition = \"2021\"\n\n[dependencies]\nfable_library_rust = { workspace = true }\ninline_colorization = { workspace = true }\n\n\n[[bin]]\nname = \"spiral_builder_d\"\npath = \"spiral_eval.rs\" ","workspace_cargo_toml_content":"[workspace]\nresol

In [ ]:
//// test
///! rust -d clap encoding encoding_rs encoding_rs_io futures-lite regex serde_json

types ()
get_command ()
|> runtime.command_get_matches_from ($'$"_ fsharp -p p -d d -a \\\"rust --deps a b=\'1\'\\\""' |> runtime.split_args)
|> run
|> async.block_on
|> resultm.unwrap'
|> _assert_eq (
    ;[
        "cargo_toml_content",
        get_cargo_toml_content { hash = "d"; deps = "a=\"*\"\nb=\'1\'" }

        "workspace_cargo_toml_content",
        get_workspace_cargo_toml_content ()
    ]
    |> am'.to_vec
    |> am'.vec_map' fun k, v =>
        new_pair (sm'.to_std_string k) (sm'.to_std_string v)
    |> mapm.b_tree_map_from_vec_pairs
    |> sm'.serialize
    |> resultm.unwrap'
    |> sm'.from_std_string
)

assert_eq / actual: "{"cargo_toml_content":"[package]\nname = \"spiral_builder_d\"\nversion = \"0.0.1\"\nedition = \"2021\"\n\n[dependencies]\nfable_library_rust = { workspace = true }\ninline_colorization = { workspace = true }\na=\"*\"\nb='1'\n\n[[bin]]\nname = \"spiral_builder_d\"\npath = \"spiral_eval.rs\" ","workspace_cargo_toml_content":"[workspace]\nresolver = \"2\"\nmembers = [\"packages/*\"]\n\n[workspace.dependencies]\nfable_library_rust = { path = \"../../../../../lib/rust/fable/fable_modules/fable-library-rust\", default-features = false, features = [\"static_do_bindings\", \"datetime\", \"guid\", \"threaded\"] }\ninline_colorization = \"~0.1\"\n\n"}" / expected: "{"cargo_toml_content":"[package]\nname = \"spiral_builder_d\"\nversion = \"0.0.1\"\nedition = \"2021\"\n\n[dependencies]\nfable_library_rust = { workspace = true }\ninline_colorization = { workspace = true }\na=\"*\"\nb='1'\n\n[[bin]]\nname = \"spiral_builder_d\"\npath = \"spiral_eval.rs\" ","workspace_cargo_toml_

In [ ]:
//// test
///! rust -d chrono clap encoding encoding_rs encoding_rs_io futures-lite regex serde_json

types ()
inl temp_folder, disposable = file_system.create_temp_directory ()
inl file_name = "test.dib"
inl path = temp_folder </> file_name |> file_system.normalize_path

"#!meta\n\n{\"kernelInfo\":{\"defaultKernelName\":\"fsharp\",\"items\":[]}}\n\n#!fsharp\n\n3 - 6"
|> file_system.write_all_text path

get_command ()
|> runtime.command_get_matches_from ($'$"_ dib -p {!path}"' |> runtime.split_args)
|> run
|> async.block_on
|> resultm.unwrap'
|> __assert_string_contains false "<pre>-3 "

$'$"{!path}.html"'
|> file_system.read_all_text
|> __assert_string_contains false "\"cell-id=1\""

disposable |> use |> ignore

00:00:00 verbose #1 file_system.create_directory / dir: C:\Users\i574n\AppData\Local\Temp\!spiral_builder_2c27c9925d77917cade0213ae13057edad1d03ba0ce47ce60a4c773c91cc8f3f\20240515-1115-0630-9930-0000006e335a
00:00:00   debug #2 runtime.execute_with_options / file_name: dotnet / arguments: ["repl", "--exit-after-run", "--run", "c:/Users/i574n/AppData/Local/Temp/!spiral_builder_2c27c9925d77917cade0213ae13057edad1d03ba0ce47ce60a4c773c91cc8f3f/20240515-1115-0630-9930-0000006e335a/test.dib", "--output-path", "c:/Users/i574n/AppData/Local/Temp/!spiral_builder_2c27c9925d77917cade0213ae13057edad1d03ba0ce47ce60a4c773c91cc8f3f/20240515-1115-0630-9930-0000006e335a/test.dib.ipynb"] / options: (None, "dotnet repl --exit-after-run --run "c:/Users/i574n/AppData/Local/Temp/!spiral_builder_2c27c9925d77917cade0213ae13057edad1d03ba0ce47ce60a4c773c91cc8f3f/20240515-1115-0630-9930-0000006e335a/test.dib" --output-path "c:/Users/i574n/AppData/Local/Temp/!spiral_builder_2c27c9925d77917cade0213ae13057edad1d03b

## tests

In [ ]:
inl tests () =
    rust.run_tests [
        "verify_app", fun _ =>
            get_command () |> runtime.command_debug_assert
    ]

## main

In [ ]:
///! 

inl main (args : array_base string) =
    inl trace_state = get_trace_state_or_init None

    inl command = get_command ()
    inl arg_matches = command |> runtime.command_get_matches
    
    inl trace_level =
        arg_matches
        |> runtime.matches_get_one (get_args () .trace_level |> fst)
        |> optionm'.unbox
        |> optionm.map sm'.from_std_string

    inl trace_state_level = trace_state.level
    match trace_level with
    | Some trace_level =>
        inl trace_state_level = trace_state.level
        trace_state_level <- trace_level |> of_string
    | None => ()

    trace Debug
        fun () => $'$"spiral_builder.main"'
        fun () => $'$"args: {!args} / {!_locals ()}"'

    inl result =
        arg_matches
        |> run
        |> async.block_on
        |> resultm.unwrap'

    if *trace_state_level = Info
    then result |> console.write_line

    0i32

inl main () =
    types ()
    $"let tests () = !tests ()" : ()
    $"let main args = !main args" : ()